# CS545: Machine Learning
## Fall 2019 - Final Project

Brent Staab and Seth Hughes

## Import code needed in this notebook

In [1]:
import numpy as np
import pandas
import torch
import copy

import os
import time

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

## DATA

##### Download the data

In [2]:
# I'm not sure why this doesn't work.  When I run the following two commands, I get the
# output at the bottom of this cell.  I had to manually download from the browser and
# unzip for this notebook to work.

#!curl -O https://github.com/chadwickbureau/baseballdatabank/archive/v2019.2.zip
#!unzip -o v2019.2.zip

#---------------------------------------------------------------------------------
#   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
#                                  Dload  Upload   Total   Spent    Left  Speed
# 100   137    0   137    0     0    193      0 --:--:-- --:--:-- --:--:--   193
# Archive:  v2019.2.zip
#   End-of-central-directory signature not found.  Either this file is not
#   a zipfile, or it constitutes one disk of a multi-part archive.  In the
#   latter case the central directory and zipfile comment will be found on
#   the last disk(s) of this archive.
# unzip:  cannot find zipfile directory in one of v2019.2.zip or
#         v2019.2.zip.zip, and cannot find v2019.2.zip.ZIP, period.

#### Import data from files

In [3]:
# Get a list of all players
# Note: We only need the first column ('playerID') from this data set
df_players = pandas.read_csv('baseballdatabank-2019.2/baseballdatabank-2019.2/core/People.csv')
df_players = df_players['playerID']

# Get batting information for all players
# Note: For records with missing data, pandas inserts 'NaN', replace
#       these values with a 0 for downstream processing.
df_batting = pandas.read_csv('baseballdatabank-2019.2/baseballdatabank-2019.2/core/Batting.csv')
df_batting.fillna(0, inplace=True)

# Get fielding information for all players
# Note: For records with missing data, pandas inserts 'NaN', replace
#       these values with a 0 for downstream processing.
# Note: There may be more than one entry for a specific combination of
#       'playerID' and 'yearID'.  This occurs if a player played more than
#       one position
df_fielding = pandas.read_csv('baseballdatabank-2019.2/baseballdatabank-2019.2/core/Fielding.csv')
df_fielding.fillna(0, inplace=True)

# Get pitching information for all players
# Note: For records with missing data, pandas inserts 'NaN', replace
#       these values with a 0 for downstream processing.
# Note: There may be more than one entry for a specific combination of
#       'playerID' and 'yearID'.  This occurs if a player played more than
#       one position
df_pitching = pandas.read_csv('baseballdatabank-2019.2/baseballdatabank-2019.2/core/Pitching.csv')
df_pitching.fillna(0, inplace=True)

# Get batting information for all players
# The db has records of anyone (players & coaches) who received votes.
# We only want players who were voted in
df_hof = pandas.read_csv('baseballdatabank-2019.2/baseballdatabank-2019.2/core/HallOfFame.csv')
df_hof.fillna(0, inplace=True)
df_hof = df_hof.loc[(df_hof['inducted'] == 'Y') & (df_hof['category'] == 'Player')]
df_hof = df_hof['playerID']
hof_list = list(df_hof.unique())

#### Generate career statistics

###### Pitching Statistics

In [4]:
# Create new dataframe to hold career stats for pitchers
# A pitcher is anyone showing up in the 'df_pitching' dataframe, which was imported above
df_career_pitching = pandas.DataFrame()

if os.path.isfile('df_career_pitching.csv'):
    df_career_pitching = pandas.read_csv('df_career_pitching.csv')
else:
    start_time = time.time()
    
    # Build data set for each pitcher in the pitching database
    count = 0
    for player in df_pitching['playerID'].unique():
        # Set variable to '1' if player is in the HOF, else set to '0'
        in_hof = 1 if player in hof_list else 0
    
        # Get all pitching statistics for this player and sum
        ds_pitch = df_pitching.loc[df_batting['playerID'] == player].sum()
        
        # Get the fielding statistics, only when they were a pitcher, for this player and sum
        ds_field = df_fielding.loc[(df_fielding['playerID'] == player) & (df_fielding['POS'] == 'P')].sum()
        
        # Determine the number of years played (should be the same in both sets, but who knows)
        pitch_yrs = ds_pitch['stint']
        field_yrs = ds_field['stint']
        max_yrs = max(pitch_yrs, field_yrs)
        
        # Get rid of the fields we don't want in both data sets
        ds_pitch.drop(['yearID', 'teamID'], inplace=True, errors='ignore')
        ds_field.drop(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS', 'WP'], inplace=True, errors='ignore')
        
        # Overwrite specific fields
        ds_pitch['playerID'] = player
        ds_pitch['stint'] = max_yrs
        ds_pitch['ERA'] = ds_pitch['ERA'] / pitch_yrs
        
        # Add 'HOF' field to data set
        ds_field['HOF'] = in_hof
        
        # For reference, these are the columns for the combined set of data
        # df_career_pitching.columns
        # Index([ 'A',    'BAOpp',   'BB', 'BFP', 'BK',  'CG', 'CS',  'DP',      'E',      'ER', 'ERA',
        #         'G',       'GF', 'GIDP',  'GS',  'H', 'HBP', 'HR', 'IBB', 'IPouts', 'InnOuts',
        #         'L',       'PB',   'PO',   'R', 'SB',  'SF', 'SH', 'SHO',     'SO',      'SV',   'W', 
        #        'WP',       'ZR', 'playerID', 'stint', 'HOF'],dtype='object')
        
        # Append fielding data to batting data and append to the dataframe
        df_career_pitching = df_career_pitching.append(ds_pitch.append(ds_field), ignore_index=True)
        count += 1
    
    # The order of the columns gets messed up, so fix at the end
    # Move 'stint' to the front, then 'playerID' so we'll get 'playerID' 'stint' ...
    # Move 'HOF' to the end of the list so we'll get ... 'HOF'
    df_career_pitching.drop(['Unnamed: 0'], axis=1, errors='ignore')
    col_name_lst = list(df_career_pitching.columns)
    col_name_lst.insert(0, col_name_lst.pop(col_name_lst.index('stint')))
    col_name_lst.insert(0, col_name_lst.pop(col_name_lst.index('playerID')))
    col_name_lst.append(col_name_lst.pop(col_name_lst.index('HOF')))
    df_career_pitching = df_career_pitching[col_name_lst]

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'It took {elapsed_time} seconds to process the pitching data. count({count})')
    
    # Save to file so we only need to generate once    
    df_career_pitching.to_csv('df_career_pitching.csv')

It took 357.3989112377167 seconds to process the pitching data. count(9655)


###### Fielding Statistics

In [5]:
# Create new dataframe to hold career stats for fielders
# A fielder is anyone showing up in the 'df_fielding' dataframe 
# AND NOT in the 'df_pitching' dataframe, both were imported above
df_career_fielder = pandas.DataFrame()

if os.path.isfile('df_career_fielder.csv'):
    df_career_fielder = pandas.read_csv('df_career_fielder.csv')
else:
    start_time = time.time()
    
    # Get a list of pitchers, these will be excluded from processing
    pitchers = list(df_pitching['playerID'].unique())
    
    # Build data set for each player in the fielding database
    count = 0
    for player in df_fielding['playerID'].unique():
        
        # If the player is a pitcher, exclude them from this dataset
        if player in pitchers:
            continue
        
        # Set variable to '1' if player is in the HOF, else set to '0'
        in_hof = 1 if player in hof_list else 0
        
        # Get all batting and fielding information for this player and sum
        ds_bat = df_batting.loc[df_batting['playerID'] == player].sum()
        ds_fld = df_fielding.loc[df_fielding['playerID'] == player].sum()
        
        # Determine the number of years played (should be the same in both sets, but who knows)
        bat_yrs = ds_bat['stint']
        fld_yrs = ds_fld['stint']
        max_yrs = max(bat_yrs, fld_yrs)
        
        # Get rid of the fields we don't want
        ds_bat.drop(['yearID', 'teamID', 'lgID', 'POS', 'G'], inplace=True, errors='ignore')
        ds_fld.drop(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS', 'SB', 'CS'], inplace=True, errors='ignore')
        
        # Overwrite specific fields
        ds_bat['playerID'] = player
        ds_bat['stint'] = max_yrs
        
        # Add 'HOF' field to data set
        ds_fld['HOF'] = in_hof
        
        # Append fielding data to batting data and append to the dataframe
        df_career_fielder = df_career_fielder.append(ds_bat.append(ds_fld), ignore_index=True)
        
        count += 1
    
    # The order of the columns gets messed up, so fix at the end
    # Move 'stint' to the front, then 'playerID' so we'll get 'playerID' 'stint' ...
    # Move 'HOF' to the end of the list so we'll get ... 'HOF'
    col_name_lst = list(df_career_fielder.columns)
    col_name_lst.insert(0, col_name_lst.pop(col_name_lst.index('stint')))
    col_name_lst.insert(0, col_name_lst.pop(col_name_lst.index('playerID')))
    col_name_lst.append(col_name_lst.pop(col_name_lst.index('HOF')))
    df_career_fielder = df_career_fielder[col_name_lst]
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'It took {elapsed_time} seconds to process the fielder data. count({count})')

    # Save to file so we only need to generate once    
    df_career_fielder.to_csv('df_career_fielder.csv')

It took 233.62158155441284 seconds to process the fielder data. count(9574)


#### Summary of career statistics

In [6]:
num_fielders = df_career_fielder.count()[0]
num_pitchers = df_career_pitching.count()[0]
num_players = num_fielders + num_pitchers
num_hof = df_hof.count()
pct_hof = (num_hof / num_players) * 100

hofers = set(df_hof.unique())
pitchers = set(df_career_pitching['playerID'].unique())
fielders = set(df_career_fielder['playerID'].unique())
num_hof_pitchers = len(pitchers.intersection(hofers))
pct_hof_pitchers = (num_hof_pitchers / num_pitchers) * 100
num_hof_fielders = len(fielders.intersection(hofers))
pct_hof_fielders = (num_hof_fielders / num_fielders) * 100

print('-----------------Summary--------------------------')
print(f'Total number of pitchers - {num_pitchers}')
print(f'Total number of fielders - {num_fielders}')
print(f'Total number of players - {num_players}')
print('--------------------------------------------------')
print(f'Number of pitchers in HOF ({num_hof_pitchers}) or  ({pct_hof_pitchers:0.3f})%')
print(f'Number of fielders in HOF ({num_hof_fielders}) or  ({pct_hof_fielders:0.3f})%')
print(f'Total in HOF ({num_hof}) or ({pct_hof:0.3f})%')

-----------------Summary--------------------------
Total number of pitchers - 9655
Total number of fielders - 9574
Total number of players - 19229
--------------------------------------------------
Number of pitchers in HOF (96) or  (0.994)%
Number of fielders in HOF (134) or  (1.400)%
Total in HOF (256) or (1.331)%


These statistics show how hard it is for a player to make the hall of fame.  This also means that the data set is exteremely unbalanced where there are more players who do NOT make the HOF than do.

## NeuralNetwork